In [ ]:
%pip install -r requirements.txt

### Configuring the Language Model (LLM)

Here, we define the configuration for the language model we want to use. This dictionary (`gemma`) specifies:
- **model**: The specific model to use (`gemma-2-2b-it`), along with its configuration identifier.
- **base_url**: The URL of the local server hosting the model API.
- **api_key**: The API key needed for authentication with the model server.
- **price**: An optional field to specify the cost per 1,000 tokens (for both input and output tokens).
- **cache_seed**: Set to `None` to disable caching, ensuring that each request is processed freshly without using previous results.

In [ ]:
gemma = {
    "config_list": [
        {
            "model": "lmstudio-community/gemma-2-2b-it-GGUF/gemma-2-2b-it-Q4_K_M.gguf:0",
            "base_url": "http://localhost:1234/v1",
            "api_key": "lm-studio",
            "price": [0.001, 0.001],  # Add custom pricing here (per 1k prompt/completion tokens)
        },
    ],
    "cache_seed": None,  # Disable caching.
}

### Initializing Conversable Agents

**Literature and Writing Expert (Agent 1)**
- **Role:** Responsible for the narrative structure, style, and overall coherence of the storybook.
- **Skills:** Creative writing, storytelling, narrative design.

**Game Theory Expert (Agent 2)**
- **Role:** Introduces strategic elements, conflicts, and decision-making scenarios in the story.
- **Skills:** Game theory, strategic analysis, conflict resolution.

**Ottoman History Expert (Agent 3)**
- **Role:** Provides historical context, cultural accuracy, and authentic details related to Ottoman history.
- **Skills:** Historical research, cultural expertise, accuracy in historical depiction.

In [ ]:
import os
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json, GroupChatManager, GroupChat

# Define agents
lit_expert = AssistantAgent(
    name="lit_expert",
    system_message="You are an expert in literature and writing, focusing on narrative structure and coherence.",
    llm_config=gemma
)

game_theorist = AssistantAgent(
    name="game_theorist",
    system_message="You are an expert in game theory, introducing strategic elements and conflicts.",
    llm_config=gemma
)

ottoman_historian = AssistantAgent(
    name="ottoman_historian",
    system_message="You are an expert in Ottoman history, ensuring historical accuracy and cultural details.",
    llm_config=gemma
)

story_writer = AssistantAgent(
    name="story_writer",
    system_message="You are responsible for writing the story based on feedback and contributions from other agents.",
    llm_config=gemma
)

user_proxy = UserProxyAgent(
    name="user_proxy",
    system_message="You are the project manager coordinating the creation of a storybook without hardcoding messages.",
    code_execution_config={
        "work_dir": "book",
        "use_docker": False,
    }
)

In [ ]:
# Define state transition logic
def state_transition(last_speaker, groupchat):
    """
    Determines the next speaker based on the current context and roles in the group chat.

    Parameters:
    last_speaker (Agent): The agent who sent the last message.
    groupchat (GroupChat): The group chat instance.

    Returns:
    Agent or None: The next speaker or None to end the conversation.
    """
    messages = groupchat.messages

    # Default to "user_proxy" if no previous messages
    if not messages:
        return user_proxy

    last_message = messages[-1]
    last_content = last_message["content"]

    if last_speaker == user_proxy:
        # User Proxy defines requirements, transition to relevant expert
        if "narrative structure" in last_content.lower():
            return lit_expert
        elif "game theory" in last_content.lower():
            return game_theorist
        elif "Ottoman history" in last_content.lower():
            return ottoman_historian
        else:
            # If ambiguous, start with literature expert
            return lit_expert

    elif last_speaker in [lit_expert, game_theorist, ottoman_historian]:
        # After an expert contributes, pass to story_writer
        return story_writer

    elif last_speaker == story_writer:
        # After the story writer contributes, go back to user_proxy for coordination
        return user_proxy

    # Default fallback to user_proxy if no clear transition is identified
    return user_proxy

In [ ]:
# Function to save story progress
def save_progress(story, file_name="storybook.txt"):
    """
    Saves the current progress of the story into a file in the same directory as the notebook.

    Parameters:
    story (str): The story text to save.
    file_name (str): The file name to save the progress.
    """
    # Get the directory of the current notebook
    notebook_dir = os.getcwd()  # This gets the current working directory of the notebook

    # Full path for the file
    file_path = os.path.join(notebook_dir, file_name)

    # Append the new content to the file
    with open(file_path, "a", encoding="utf-8") as file:
        file.write(story + "\n\n")

    print(f"Progress saved to {file_path}")

In [ ]:
# Create GroupChat
groupchat = GroupChat(
    agents=[lit_expert, game_theorist, ottoman_historian, story_writer, user_proxy],
    messages=[],
    max_round=20,
    speaker_selection_method=state_transition,
)

# GroupChat Manager
manager = GroupChatManager(groupchat=groupchat, llm_config=gemma)

# Initiate chat
user_proxy.initiate_chat(
    manager, message="Create a storybook that combines elements of literature, game theory, and Ottoman history."
)

# Simulate conversation and save after story_writer contributes
while not groupchat.is_finished():
    manager.step()  # Proceed to the next speaker

    # Check if story_writer contributed and save the progress
    last_message = groupchat.messages[-1]
    if last_message["sender"] == "story_writer":
        save_progress(last_message["content"])

# Final save
last_story = "\n".join([msg["content"] for msg in groupchat.messages if msg["sender"] == "story_writer"])
save_progress(last_story)